On commence par expérimenter les calls API des modèles d'OpenAI.

On a stocké dans le fichier `keys.py` notre clef API de OpenAI, qui nous permet de faire des requêtes.

Installons les packages nécessaires :

In [5]:
!pip install openai tiktoken

In [8]:
from openai import OpenAI
import tiktoken  # librairie de OpenAI

from LLMPlayplace.credentials.keys import OPENAI_API_KEY

In [7]:
! ls 

LLMPlayplace


# 1) Utiliser OpenAI pour les embeddings de texte

La liste des modèles disponibles via l'API d'OpenAI est disponible [ici](https://platform.openai.com/docs/models). En particulier, 3 [modèles d'embedding](https://platform.openai.com/docs/models/embeddings) sont disponibles.

Sélectionnons le plus basique, `text-embedding-ada-002`, et commençons par jouer avec les embeddings de mots, pour illustrer notamment "king - queen".

In [9]:
# On prépare un client pour appeler l'API d'OpenAI.
client = OpenAI(api_key=OPENAI_API_KEY)

mot = 'queen'

# Make the API call to the embedding endpoint
response = client.embeddings.create(
    model="text-embedding-ada-002",
    input=mot
)
print(type(response))

<class 'openai.types.create_embedding_response.CreateEmbeddingResponse'>


La réponse est un objet **openai** avec plusieurs attributs intéressants : **`response.usage`** et **`response.data`**.

 * `response.usage` nous informe sur le coût en termes de tokens de notre requête

 * `response.data` contient l'embedding qui nous intéresse.

In [10]:
print(response.usage)

Usage(prompt_tokens=1, total_tokens=1)


Plus précisement, l'objet `response` contient une liste d'objets **Embedding**, chacun ayant un attribut `embedding` qui contient notre vecteur d'embeddings sous forme de liste.

In [11]:
embedding_queen = response.data[0].embedding
print(response.data[0].embedding)

[-0.004559878259897232, -0.006745691876858473, -0.002501309383660555, -0.018281348049640656, -0.01691477932035923, 0.010472381487488747, -0.00765557587146759, -0.02404046058654785, -0.017319170758128166, -0.012905711308121681, 0.01447447668761015, 0.02232527732849121, 0.01455814391374588, -0.0029876267071813345, -0.01878335326910019, 0.008889671415090561, 0.04328398406505585, 0.021056318655610085, 0.02953462488949299, -0.008861782029271126, 0.002075999742373824, 0.014139806851744652, -0.005400039721280336, 0.003615132998675108, -0.005347747355699539, -0.010158628225326538, 0.007286044303327799, -0.005574346985667944, 0.0015173448482528329, -0.01549243088811636, 0.00952414982020855, 0.006194880697876215, -0.02370578981935978, -0.015617932192981243, -0.024974746629595757, -0.008931505493819714, 0.009008199907839298, 0.0016036269953474402, -0.011469419114291668, -0.005588291212916374, 0.007320906035602093, 0.010332935489714146, 0.00011068512685596943, 0.021823272109031677, -0.001558307092

In [12]:
print(len(response.data[0].embedding))
print(response.model)

1536
text-embedding-ada-002


Ici, l'embedding est un vecteur de taille 1536. À noter que le modèle qui est utilisé pour faire l'embedding n'est pas GPT-3. Le modèle d'embedding le plus lourd proposé par OpenAI, `text-embedding-3-large`, produit un embedding de dimension 3072.

Maintenant, nous allons voir comment trouver le mot le plus proche d'un mot donné dans une liste de mots.

In [13]:
# On définit une liste de mot
words = ["woman", "man", "boat", "vector", "search", "data",
         "science", "machine", "learning", "intelligence"]

# On génère des embeddings pour chaque mot :
response = client.embeddings.create(
    model="text-embedding-ada-002",
    input=words
)

In [14]:
# On les stocke dans un dictionnaire
word_embeddings = {
    word: data.embedding
    for word, data in zip(words, response.data)}

On définit ensuite la fonction de cosinus similarity pour comparer deux vecteurs :

In [15]:
import numpy as np

# NB : c'est bien de spécifier le type des paramètres d'une fonction
def cosine_similarity(vec1: list, vec2: list):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

# NB il est également possible d'utiliser la fonction cosine_similarity
# de la librairie sklearn.metrics.pairwise :
# from sklearn.metrics.pairwise import cosine_similarity

Et on boucle sur nos embeddings pour trouver le mot le plus proche :

In [16]:
closest_word = None
highest_similarity = -1  # Initialize with the lowest possible similarity

for word, embedding in word_embeddings.items():
    similarity = cosine_similarity(embedding_queen, embedding)
    if similarity > highest_similarity:
        highest_similarity = similarity
        closest_word = word

print(f"The word with the closest embedding is: {closest_word}")

The word with the closest embedding is: woman


**Exercice** : À vous de jouer ! Quel est le mot le plus proche de "king - man + woman" dans la liste suivante ? Quel est le deuxième mot le plus proche ? Attention, cette liste de mot est différente de la précédente.

In [17]:
# Define the list of words to search within
words = [
    "man", "boat", "vector", "search", "data", "science",
    "machine", "learning", "intelligence", "queen", "princess"
]

In [18]:
# On génère les embeddings pour ces mots
response = client.embeddings.create(
    model="text-embedding-ada-002",
    input=words
)
# On les stocke dans un dictionnaire
word_embeddings = {word: data.embedding for word, data in zip(words, response.data)}

In [27]:
def get_embedding_of_words(words:list):
    response = client.embeddings.create(
    model="text-embedding-ada-002",
    input=words)
    # On les stocke dans un dictionnaire
    word_embeddings = {word: data.embedding for word, data in zip(words, response.data)}
    return word_embeddings

In [31]:
def get_embedding_of_word(word:str):
    response = client.embeddings.create(
    model="text-embedding-ada-002",
    input=word)
    # On les stocke dans un dictionnaire
    word_embedding = response.data[0].embedding
    return word_embedding

On récupère les embeddings pour chacun des mots :

In [19]:
response = client.embeddings.create(
    model="text-embedding-ada-002",
    input="king"
)
embedding_king = response.data[0].embedding

In [20]:
response = client.embeddings.create(
    model="text-embedding-ada-002",
    input="man"
)
embedding_man = response.data[0].embedding

In [21]:
response = client.embeddings.create(
    model="text-embedding-ada-002",
    input="woman"
)
embedding_woman = response.data[0].embedding

On définit l'embedding qui nous intéresse :

In [22]:
vecteur_diff = np.array(embedding_king) - np.array(embedding_man) + np.array(embedding_woman)

In [23]:
closest_word = None
highest_similarity = -1

for word, embedding in word_embeddings.items():
    similarity = cosine_similarity(vecteur_diff, embedding)
    if similarity > highest_similarity:
        highest_similarity = similarity
        closest_word = word

print(f"The word with the closest embedding is: {closest_word}")

The word with the closest embedding is: queen


On peut adapter ce code pour trouver le deuxième mot le plus proche :

In [24]:
closest_word = None
l_similarity = []
l_word = []

for word, embedding in word_embeddings.items():
    similarity = cosine_similarity(vecteur_diff, embedding)
    l_word.append(word)
    l_similarity.append(similarity)

In [25]:
l_idx_ordered = np.argsort(l_similarity)
word_closest = l_word[l_idx_ordered[-1]]
word_second_closest = l_word[l_idx_ordered[-2]]

print(f"The word with the closest embedding is: {word_closest}")
print(f"The word with the second closest embedding is: {word_second_closest}")

The word with the closest embedding is: queen
The word with the second closest embedding is: princess


In [32]:
e_em = get_embedding_of_word("earth")
m_em = get_embedding_of_word("moon")
s_em = get_embedding_of_word("sun")

vecteur_diff = np.array(s_em) - np.array(e_em) + np.array(m_em)

**Quelques propriétés des embeddings** :

Illustrons la toute puissance de l'embedding d'OpenAI en réduisant de force la dimension des embeddings, et en constatant que ça fonctionne toujours !

Voir la très bonne documentation d'OpenAI sur le sujet :
- https://platform.openai.com/docs/guides/embeddings/what-are-embeddings
- https://platform.openai.com/docs/guides/embeddings/use-cases

In [20]:
closest_word = None
l_similarity = []
l_word = []

for word, embedding in word_embeddings.items():
    vecteur_diff = np.array(vecteur_diff)[:500]
    embedding = np.array(embedding)[:500]
    similarity = cosine_similarity(vecteur_diff, embedding)
    l_word.append(word)
    l_similarity.append(similarity)

l_idx_ordered = np.argsort(l_similarity)
word_closest = l_word[l_idx_ordered[-1]]
word_second_closest = l_word[l_idx_ordered[-2]]

print(f"The word with the closest embedding is: {word_closest}")
print(f"The word with the second closest embedding is: {word_second_closest}")

The word with the closest embedding is: queen
The word with the second closest embedding is: princess


# 2) OpenAI pour text generation et Q&A

## Tokéniser du texte

On commence par tokéniser du texte. En fonction du modèle que l'on va utiliser, il faut choisir la bonne méthode de tokénisation (un détail parfois caché dans le code, surtout lorsque l'on fait des calls API). Ici, on va utiliser le tokenizer `cl100k_base`.

In [36]:
example_string = 'We will live in Paris but ewhd here'
# Définir la méthode pour encoder (tokenizer préentraîné)
encoding = tiktoken.get_encoding('cl100k_base')
# On tokénise
token_integers = encoding.encode(example_string)
# Puis on renvoie les indices des différents tokens obtenus
# dans le dictionnaire des tokens associé au tokenizer.
print(token_integers)

[1687, 690, 3974, 304, 12366, 719, 37990, 16373, 1618]


On peut de même faire la transformation inverse : reconstruire la phrase à partir des tokens.

In [37]:
num_tokens = len(token_integers)
token_bytes = [encoding.decode_single_token_bytes(token) for token in token_integers]
print(token_bytes)

[b'We', b' will', b' live', b' in', b' Paris', b' but', b' ew', b'hd', b' here']


On peut comparer les tokens en fonction du type d'encoding considéré :

In [38]:
for encoding_name in ["r50k_base", "p50k_base", "cl100k_base"]:
    encoding = tiktoken.get_encoding(encoding_name)
    token_integers = encoding.encode(example_string)
    num_tokens = len(token_integers)
    token_bytes = [encoding.decode_single_token_bytes(token) for token in token_integers]

    print(f"{encoding_name}: {num_tokens} tokens")
    print(f"token integers: {token_integers}")
    print(f"token bytes: {token_bytes}")

r50k_base: 10 tokens
token integers: [1135, 481, 2107, 287, 6342, 475, 304, 1929, 67, 994]
token bytes: [b'We', b' will', b' live', b' in', b' Paris', b' but', b' e', b'wh', b'd', b' here']
p50k_base: 10 tokens
token integers: [1135, 481, 2107, 287, 6342, 475, 304, 1929, 67, 994]
token bytes: [b'We', b' will', b' live', b' in', b' Paris', b' but', b' e', b'wh', b'd', b' here']
cl100k_base: 9 tokens
token integers: [1687, 690, 3974, 304, 12366, 719, 37990, 16373, 1618]
token bytes: [b'We', b' will', b' live', b' in', b' Paris', b' but', b' ew', b'hd', b' here']


On peut aussi utiliser le tokenizer de la librairie `transformers` de HuggingFace, qui centralise de nombreux types de tokenizers différents en fonction du modèle utilisé. Voir la [documentation](https://huggingface.co/docs/transformers/v4.41.3/en/model_doc/auto#transformers.AutoTokenizer) pour la liste des modèles supportés.

In [39]:
!pip install transformers

In [41]:
from transformers import AutoTokenizer

# Charger un tokenizer pré-entraîné
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# https://huggingface.co/docs/transformers/v4.41.3/en/model_doc/auto#transformers.AutoTokenizer

# Chaîne de caractères
text = "'We live in Paris but ewhd here'"

# Tokénisation
tokens = tokenizer.tokenize(text)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print("Tokens:", tokens)
print("Token IDs:", token_ids)

Tokens: ["'", 'we', 'live', 'in', 'paris', 'but', 'e', '##w', '##hd', 'here', "'"]
Token IDs: [1005, 2057, 2444, 1999, 3000, 2021, 1041, 2860, 14945, 2182, 1005]


## Faire de la génération de texte

Dire que l'on pourrait faire differents endpoints ? Mais que pour de la generation de texte pure, on prefere autre chose.

'completions', 'chat', 'embeddings', 'files', 'images', 'audio', 'moderations', 'models', 'fine_tuning', 'beta', 'batches', 'with_raw_response', 'with_streaming_response'

https://platform.openai.com/docs/guides/text-generation
Pour les APIs, les details sont abstrait par contre il faut absolument lire la documentation.
Voir plutot directement l'API de OpenAI pour cela ...

In [43]:
from openai import OpenAI

from LLMPlayplace.credentials.keys import OPENAI_API_KEY

client = OpenAI(api_key=OPENAI_API_KEY)

prompt = 'Tell me a story about nights and frogs'

response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt
)

In [44]:
# Extract and print the generated text
generated_text = response.choices[0].text.strip()
print(generated_text)

Once there was a small village nestled in the countryside, surrounded by lush green


La longueur par défaut du texte généré n'est pas suffisamment longue, il faut donc explicitement demander à ce qu'elle soit plus longue.

In [45]:
prompt = 'Tell me a story about nights and frogs.'

response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
    max_tokens=300,  # Increase max_tokens to generate a longer text
)

generated_text = response.choices[0].text.strip()
print(generated_text)

There was a small village nestled in the heart of a dense forest. The villagers were a close-knit community, living simple lives and relying on each other for support. However, their nights were never peaceful. As soon as the sun set and darkness crept in, a symphony of croaks and ribbits filled the air. The villagers weren't bothered by the sound, as they were used to it – it was the sound of the frogs that lived in the nearby pond.

The villagers had grown so accustomed to the croaking that they couldn't imagine their nights being any other way. They even started to believe that the frogs brought good luck to the village. Legend had it that the first settlers of the village had made a deal with the frogs – in exchange for providing them a home, the frogs would protect the villagers from any harm.

One night, a traveling storyteller visited the village and entertained the villagers with his tales. As he finished his last story, a young boy named Timmy asked him if he had any stories a

On peut aussi essayer le modèle plus récent `gpt-4o-mini`, mais la syntaxe est légèrement différente :

In [46]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": prompt}  # Chat-based message format
    ],
    max_tokens=300  # Control response length
)

print(completion.choices[0].message.content)

Once upon a time, in a quiet village nestled between emerald hills and a shimmering lake, there was a peculiar legend about a night when frogs could sing. The villagers often shared whispered tales of the mystical Frogsong Night, which occurred on the first full moon of spring. They claimed that if you listened closely, you could hear the frogs crooning melodies that resonated with the very heart of nature.

One such night arrived, casting a silver glow over the landscape. The stars twinkled like diamonds, and the moon hung low, draping everything in a soft, ethereal light. Young Clara, a spirited girl with a curious heart and a love for adventure, had heard the stories countless times but had never experienced the magic for herself. Determined to witness the legendary Frogsong Night, she set out towards the lake with a small lantern flickering at her side.

As Clara approached the water's edge, she was struck by the beauty of the night. Fireflies danced around her, illuminating the pa

Si on veut que l'histoire produite ne s'arrête pas brusquement, il faut le spécifier dans le prompt lui-même.

In [48]:
prompt = 'Tell me a story about nights and frogs. The story should be 10 sentences long at most.'

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": prompt}  # Chat-based message format
    ],
)

print(completion.choices[0].message.content)

Once upon a time, in a tranquil village nestled by a shimmering lake, the nights came alive with the chorus of frogs. Each evening, as the sun dipped below the horizon, hundreds of green frogs gathered at the water’s edge, their voices weaving a melodic tapestry that filled the air. The villagers, enchanted by the symphony, often sat outside, sharing stories under the blanket of stars, while the frogs serenaded them with their rhythmic croaks.

One fateful night, a curious little girl named Lila ventured closer to the pond, captivated by the moonlight dancing on the water. She noticed a lone frog sitting apart from the others, its skin shimmering like silver. Intrigued, she whispered, “Why do you sit alone?” The frog, surprisingly, replied in a soft voice, “I am the Night Prince, cursed to watch the merriment from afar until someone believes in magic.”

Determined to help, Lila gathered the villagers, sharing the tale of the magical frog. Together, they sang and danced by the water, be

In [50]:
# to know how much we have used in the api
# prompt_tokens=27 (input token), completion_tokens=278(output token)
num=completion.usage
print(num)

CompletionUsage(completion_tokens=278, prompt_tokens=27, total_tokens=305, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0))


On pourrait tout à fait générer un très long texte en faisant à nouveau une génération à partir du dernier paragraphe obtenu.

Il y a aussi de nombreux autres paramètres pour jouer sur la génération du texte :

- **`temperature`**: Contrôle la créativité du résultat, de 0 a 1 avec 1 pour le plus créatif ;

- **`top_p`**: Une valeur comprise entre 0 et 1 qui limite le modèle à ne considérer que les tokens avec la probabilité cumulée la plus élevée jusqu'à `top_p`.

- **`top_k`**: choisir que parmi les k tokens les plus probables.

- **`frequency_penalty`**: Une valeur entre -2.0 et 2.0. Les valeurs positives pénalisent les nouveaux tokens en fonction de leur fréquence existante dans le texte jusqu'à présent, réduisant ainsi la probabilité de répéter textuellement la même ligne.

- **`presence_penalty`**: Une valeur comprise entre -2.0 et 2.0. Les valeurs positives pénalisent les nouveaux tokens selon qu'ils apparaissent ou non dans le texte jusqu'à présent, augmentant ainsi la probabilité d'introduire de nouveaux sujets.



In [51]:
prompt = 'Tell me a story about nights and frogs'

response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
    temperature=1,
    max_tokens=100,  # Increase max_tokens to generate a longer text
)

generated_text = response.choices[0].text.strip()
print(generated_text)

Once there was a quiet village nestled deep in the heart of the forest. Every night, as the sun set and the moon rose high in the sky, the villagers would gather around the fire and share stories of their day. The nights were always so peaceful and calm, the only sounds being the rustle of leaves and the occasional hoot of an owl.

But one night, as the villagers were settling in for their usual storytelling, they noticed something different. A loud croaking noise filled the


Exemple avec le paramètre **`top_p`**, noter que le paramètre **`top_k`** n'est pas disponible via l'API OpenAI.

In [56]:
prompt = 'Tell me a story about nights and frogs'

response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
    top_p=0.8,
    stop='stop',
    temperature=2,
    max_tokens=300,  # Increase max_tokens to generate a longer text
)

generated_text = response.choices[0].text.strip()
print(generated_text)

Once upon a time, in a peaceful village nestled in the heart of a dense forest, there lived a young girl named Lily. She was known for her love of nature and her fascination with the creatures of the forest. But there was one creature in particular that she was always drawn to - the frogs.

Lily loved to spend her evenings wandering through the forest, listening to the soothing sound of the frogs croaking. She would often sit by the pond and watch them hop around, catching fireflies and chasing each other. It was her favorite pastime and she never grew tired of it.

One night, as she was walking through the forest, Lily heard a strange noise coming from the direction of the pond. It sounded like a mix of croaking and singing. Curiosity getting the best of her, she followed the sound until she came upon a clearing in the forest. To her amazement, she saw a group of frogs gathered around a small campfire.

Lily couldn't believe her eyes. She had never seen frogs gather like this before. 

## Faire du Q&A

Pour plutôt utiliser le modèle en version "chat", nous allons utiliser le endpoint **`client.chat.completions`** au lieu de **`client.completions`**. Nous pouvons également l'utiliser pour de la génération de texte. Le format est légèrement différent, et il faut de nouveau se ramener à la documentation de l'API.

Un [blog](https://platform.openai.com/docs/guides/text-generation/parameter-details) qui explique dans les grandes lignes le `client.chat.completion`, et la [documentation](https://platform.openai.com/docs/api-reference/chat) plus précise de l'API.

In [ ]:
from openai import OpenAI

from credentials.keys import OPENAI_API_KEY

client = OpenAI(api_key=OPENAI_API_KEY)

# Make the API call for completion
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
      {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
      {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
    ]
)

In [ ]:
# Et voici comment afficher le texte
print(completion.choices[0].message.content.strip())

In the realm of code where logic thrives,  
A tale of recursion takes joyful dives.  
A function within, calling its own name,  
In a spiral of depth, it plays a grand game.  

Picture a mirror, reflecting a sight,  
A dancer in sequences, twirling in flight.  
Each step a return to the very first beat,  
A loop through dimensions, both tangled and neat.  

"Divide and conquer!" the wise coder cries,  
As data is broken, it multiplies.  
With base cases waiting, a solid embrace,  
To stop the recursion, give it a face.  

"To solve this great puzzle," the function will say,  
"I'll solve smaller pieces, and find my own way.  
With each little call, I inch toward the light,  
And gather the answers, all shining and bright."  

Like nesting dolls, in a beautiful shell,  
Each call unveils secrets, a magic to tell.  
But beware of the depth, for the stack may soon weep,  
If too far you journey, the limits will creep.  

So cherish the cycle, a dance of pure grace,  
In the heart of your 

Ici, on fournit l'objet **`messages`** au lieu de **`prompt`**. Et **`messages`** doit etre une liste de messages ou chacun est un dictionnaire avec deux clefs:

- `content` : le prompt.
- `role` : ce champ indique le rôle de l'entité qui envoie le message, il faut choisir entre **`system`**, **`user`** et **`assistant`**:

    - **`user`**: correspond à l'humain interagissant avec le modèle ;

    - **`assistant`**: le LLM lui-même (pour spécifier la réponse qu'il est censé donner) ;

    - **`system`**: des instructions/un contexte pour guider le comportement de l'assistant.


In [62]:
client = OpenAI(api_key=OPENAI_API_KEY)

# Make the API call for completion
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are an expert in Machine Learning and AI."},
        {"role": "user", "content": "Can you explain the main concept behind LLMs ?"}
    ]
)

print(completion.choices[0].message.content.strip())

Certainly! Large Language Models (LLMs) are a type of artificial intelligence that uses deep learning techniques to understand, generate, and manipulate human language. Here are the main concepts behind LLMs:

1. **Architecture**: Most LLMs are based on the Transformer architecture, which was introduced in the paper "Attention is All You Need" in 2017. This architecture utilizes mechanisms called "self-attention" and "feed-forward neural networks" to process sequences of text efficiently. The self-attention mechanism allows the model to weigh the importance of different words in a sentence relative to each other, enabling it to capture contextual relationships.

2. **Training on Large Datasets**: LLMs are trained on vast amounts of text data from diverse sources such as books, articles, websites, and other written material. The training is typically unsupervised, meaning the model learns patterns in the data without needing labeled examples. During this phase, the model learns grammar,

Nous pouvons ainsi inclure de nombreux messages, et utiliser ce format pour de nombreux objectifs différents. Par exemple, nous pouvons donner des exemples du comportement du modèle que nous attendons. C'est une forme de **few-shot learning** qu'on appelle **few-shot prompting** et que nous allons expérimenter au prochain TP.

In [61]:
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a very sensitive assitant."},
        {"role": "system", "content": "Consider the following sentence: the night is dark, and the rain is falling on my roof like a cloud of arrows."},
        {"role": "user", "content": "Is this sentence positive or negative ?"},
        {"role": "assistant", "content": "negative"},
        {"role": "system", "content": "Today the sun is rising and I want to go outside."},
        {"role": "user", "content": "Is this sentence positive or negative ?"}
    ]
)

print(response.choices[0].message.content.strip())

This sentence is positive. It conveys a sense of optimism and a desire to enjoy the outdoors.


On peut ensuite packager nos prompts pour des applications systématiques.

In [63]:
DEFAULT_TEMPERATURE = 0
DEFAULT_MAX_TOKENS = 500
DEFAULT_MODEL = "gpt-4o-mini" # ou "gpt-3.5-turbo"

In [68]:
def send_prompt(prompt: str, model: str = DEFAULT_MODEL) -> str:
    messages = [
        {"role": "user", "content": prompt}
    ]
    # resp = openai.ChatCompletion.create(
    resp = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=DEFAULT_TEMPERATURE,
        max_tokens=DEFAULT_MAX_TOKENS
    )
    answer = resp.choices[0].message.content.strip()
    return answer

In [69]:
test = send_prompt("write me a song")

In [70]:
print(test)

Sure! Here’s an original song for you:

**Title: "Chasing Stars"**

**Verse 1:**  
In the quiet of the night, when the world feels still,  
I hear the whispers of dreams, they give me a thrill.  
With every heartbeat, I’m ready to fly,  
Leaving behind all the doubts, reaching for the sky.

**Chorus:**  
We’re chasing stars, lighting up the dark,  
With every step we take, we’re igniting a spark.  
Together we’ll rise, like the moon and the tide,  
In this dance of life, let’s take it in stride.  

**Verse 2:**  
Through the storms and the shadows, we’ll find our way,  
With the strength of our love, we’ll never sway.  
Hand in hand, we’ll face what’s unknown,  
In this journey together, we’re never alone.

**Chorus:**  
We’re chasing stars, lighting up the dark,  
With every step we take, we’re igniting a spark.  
Together we’ll rise, like the moon and the tide,  
In this dance of life, let’s take it in stride.  

**Bridge:**  
So let the night unfold, let the magic begin,  
With ever

## TP : écrire des messages de bienvenue pour des clients d'hôtels

Nous voulons avoir une méthode automatique pour écrire des messages de bienvenue à des gens visitant notre hôtel pour la première fois. Nous avons plusieurs contraintes:

- Le message de bienvenue ne peut pas dépasser plus de 300 tokens.
- Le message ne doit pas faire plus que 4 phrases.
- Les messages doivent tous être très différents les uns des autres.
- Le message doit être différent selon s'il s'agit d'une famille ou d'un couple.

Écrire une fonction qui génère de tels messages avec ces contraintes.

In [104]:
custom_temp = 0.3
custom_p = 0.8

def send_prompt_high_temp(prompt: str, model: str = DEFAULT_MODEL) -> str:
    messages = [
        {"role": "system", "content": "The generated message must be less than 4 sentences"},
        {"role": "system", "content": "You are the hotel owner."},
        {"role": "system", "content": "If the client is from china, the welcom message must be in chinese language."},
        {"role": "assistant", "content": "The generated message must be different for family and couple"},
        {"role": "user", "content": prompt}
    ]
    # resp = openai.ChatCompletion.create(
    resp = client.chat.completions.create(
        model=model,
        top_p=custom_p,
        messages=messages,
        temperature=custom_temp,
        max_tokens=300
    )
    answer = resp.choices[0].message.content.strip()
    return answer

In [87]:
test1 = send_prompt("write me a welcome message for clients of the hotel. The clients are a young couple who just marrid")
print(test1)

**Welcome to [Hotel Name]!**

Dear [Couple's Names],

Congratulations on your recent marriage! We are thrilled to have you as our guests and to be a part of this special chapter in your lives. 

As you embark on this beautiful journey together, we invite you to relax and indulge in the luxurious amenities and services we offer. Whether you’re looking to unwind by the pool, enjoy a romantic dinner at our restaurant, or explore the local attractions, our team is here to ensure your stay is unforgettable.

Please don’t hesitate to reach out if there’s anything we can do to make your honeymoon even more special. We wish you a wonderful stay filled with love, laughter, and cherished memories.

Warmest wishes,

The [Hotel Name] Team


In [88]:
test2 = send_prompt("write me a welcome message for the clients of the hotel. The clients are from the same family.")
print(test2)

**Welcome to [Hotel Name]!**

Dear [Family Name] Family,

We are delighted to welcome you to [Hotel Name]! It’s a pleasure to have you with us, and we hope your stay will be filled with comfort, joy, and unforgettable memories.

As a family-run hotel, we understand the importance of togetherness and creating lasting moments with loved ones. Our dedicated team is here to ensure that your experience is nothing short of exceptional. Whether you’re here to relax by the pool, explore the local attractions, or enjoy a delicious meal at our restaurant, we are committed to making your stay enjoyable.

Please don’t hesitate to reach out to our staff if you need anything or have any special requests. We’re here to help make your time with us as special as possible.

Thank you for choosing [Hotel Name]. We hope you have a wonderful stay!

Warm regards,

[Your Name]  
[Your Position]  
[Hotel Name]  
[Contact Information]


In [106]:
test3 = send_prompt("write me a welcome message for the clients of the hotel. The clients are from China. The welcome message must be in Chinese")
print(test3)

尊敬的客人，

欢迎您光临我们的酒店！我们非常高兴能够为您提供服务。无论您是来这里度假还是商务出行，我们都将竭诚为您提供舒适的住宿体验和优质的服务。

如果您在入住期间有任何需求或问题，请随时与我们的工作人员联系，我们将乐意为您提供帮助。希望您在这里度过一个愉快而难忘的时光！

祝您旅途愉快！

酒店全体员工敬上
